<a href="https://colab.research.google.com/github/Mahdi-0599/NLP-HATESPEECH/blob/master/model_cnn_hatespeech_french.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Description
Dataset contains nearly 32K tweets which are labeled based on having racist or sexist content. We are going to analyse this dataset and tweets, and by the end, create a classification model to classify tweets.   
Each row in the dataset has 3 columns:
* `id`: Assigned ID to this tweet by Analytics Vidhya.
* `label`: Tweet label, 1 if tweet has hatred content and 0 otherwise.
* `tweet`: Tweet text.  

Dataset is provided by [Analytics Vidhya](http://https://datahack.analyticsvidhya.com/contest/practice-problem-twitter-sentiment-analysis/)  

In [ ]:
import numpy as np
import pandas as pd
import nltk
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

data = pd.read_csv('../input/french-tweets/french_tweets.csv')
data.head()

,label,text
0,0,"- Awww, c'est un bummer. Tu devrais avoir davi..."
1,0,Est contrarié qu'il ne puisse pas mettre à jou...
2,0,J'ai plongé plusieurs fois pour la balle. A ré...
3,0,Tout mon corps a des démangeaisons et comme si...
4,0,"Non, il ne se comporte pas du tout. je suis en..."


In [ ]:
data.columns=['label','tweet']
data.head()

,label,tweet
0,0,"- Awww, c'est un bummer. Tu devrais avoir davi..."
1,0,Est contrarié qu'il ne puisse pas mettre à jou...
2,0,J'ai plongé plusieurs fois pour la balle. A ré...
3,0,Tout mon corps a des démangeaisons et comme si...
4,0,"Non, il ne se comporte pas du tout. je suis en..."


How are tweets spread among these 2 classes?

In [ ]:
def eval_fun(labels, preds):
    labels = label.split(' ')
    preds = tweet.split(' ')
    rr = (np.intersect1d(label, tweet))
    precision = np.float(len(rr)) / len(tweet)
    recall = np.float(len(rr)) / len(label)
    try:
        f1 = 2 * precision * recall / (precision + recall)
    except ZeroDivisionError:
        return (precision, recall, 0.0)
    return (precision, recall, f1)
print(1)

1


In [ ]:
import numpy as np
print("Hatred labeled: {}\nNon-hatred labeled: {}".format(
    (data.label == 1).sum(),
    (data.label == 0).sum()
))

Hatred labeled: 755120
Non-hatred labeled: 771604


> ## Extracting features

#### Hashtags and mentions
We'll extract hashtags for each tweet as an extra column to explore them later.   
For user mentions, all of the usernames have been replaced with `'user'` so we can't get any data from it, we'll just remove mentions and keep the number of mentions in each tweet as an extra features for that tweet.  

In [ ]:
hashtags = data['tweet'].str.extractall('#(?P<hashtag>[a-zA-Z0-9_]+)').reset_index().groupby('level_0').agg(lambda x: ' '.join(x.values))
data.loc[:, 'hashtags'] = hashtags['hashtag']
data['hashtags'].fillna('', inplace=True)

data.loc[:, 'mentions'] = data['tweet'].str.count('@[a-zA-Z0-9_]+')

data.tweet = data.tweet.str.replace('@[a-zA-Z0-9_]+', '')

#### Removing anything but the words
Now we'll remove anything but the words (punctuations, numbers, etc). Note that this time we'll replace them with a blank space since it might be a `_` or `-` or a punctuation with no space from the next word and we don't want the words to join together.  

In [ ]:
data.tweet = data.tweet.str.replace('[^a-zA-Z]', ' ')

#### Lemmatization
We lemmatize tweets' words as we have the sentences and we can tag part of speeches, and will stem hashtags.  

In [ ]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag, FreqDist, word_tokenize

stemmer = SnowballStemmer('english')
lemmer = WordNetLemmatizer()

part = {
    'N' : 'n',
    'V' : 'v',
    'J' : 'a',
    'S' : 's',
    'R' : 'r'
}

def convert_tag(penn_tag):
    if penn_tag in part.keys():
        return part[penn_tag]
    else:
        return 'n'
def tag_and_lem(element):
    sent = pos_tag(word_tokenize(element))
    return ' '.join([lemmer.lemmatize(sent[k][0], convert_tag(sent[k][1][0]))
                    for k in range(len(sent))])

#data.loc[:, 'tweet'] = data['tweet'].apply(lambda x: tag_and_lem(x))
data.loc[:, 'hashtags'] = data['hashtags'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))


bild model cnn

In [ ]:
texts = np.array(data['tweet'])
labels = np.array(data['label'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

X = data.tweet
Y = data.label
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.30)

In [ ]:
import string
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_words = 1000
max_len = 100
samples = texts.shape[0]
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
word_index = tokenizer.word_index
print(data.shape)
print(labels.shape)

(1526724, 4)
(1526724,)


In [ ]:
#from sklearn.model_selection import train_test_split
#x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.30)
#print(x_train.shape)
#print(x_val.shape)
#print(x_test.shape)

In [ ]:
#x_train = data[:7925]
#y_train = labels[:7925]

#x_val = data[7925:8925]
#y_val = labels[7925:8925]

#x_test = data[8925:]
#y_test = labels[8925:]

#print(x_train.shape)
#print(x_val.shape)
#print(x_test.shape)
##




In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, LSTM, GRU, Bidirectional
model = Sequential()
model.add(Embedding(1+len(word_index), 16))
model.add(Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.5, return_sequences=True)))
model.add(Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.5)))
model.add(Dense(1, activation='sigmoid'))
model.summary()




Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          2880192   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 64)          12544     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 2,917,633
Trainable params: 2,917,633
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
history = model.fit(sequences_matrix,Y_train,batch_size=512,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
1670/1670 [==============================] - 1699s 1s/step - loss: 0.5157 - accuracy: 0.7428 - val_loss: 0.4953 - val_accuracy: 0.7558
Epoch 2/10
1670/1670 [==============================] - 1701s 1s/step - loss: 0.4933 - accuracy: 0.7579 - val_loss: 0.4839 - val_accuracy: 0.7629
Epoch 3/10
1670/1670 [==============================] - 1713s 1s/step - loss: 0.4844 - accuracy: 0.7636 - val_loss: 0.4782 - val_accuracy: 0.7668
Epoch 4/10
1670/1670 [==============================] - 1704s 1s/step - loss: 0.4803 - accuracy: 0.7664 - val_loss: 0.4758 - val_accuracy: 0.7684
Epoch 5/10
1670/1670 [==============================] - 1701s 1s/step - loss: 0.4776 - accuracy: 0.7679 - val_loss: 0.4758 - val_accuracy: 0.7692


In [ ]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)

14314/14314 [==============================] - 751s 52ms/step - loss: 0.4764 - accuracy: 0.7695


In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.476
  Accuracy: 0.770


In [ ]:
#model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
#history = model.fit(X_train,Y_train,epochs=10,batch_size=64,validation_data=(X_test,Y_test))


In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

KeyError: 'acc'

In [ ]:
def test(model):
    f1_score = lambda precision, recall: 2 * ((precision * recall) / (precision + recall))
    nexamples, recall, precision = model.test('fasttext.test')
    print (f'recall: {recall}' )
    print (f'precision: {precision}')
    print (f'f1 score: {f1_score(precision,recall)}')
    print (f'number of examples: {nexamples}')
